<a href="https://colab.research.google.com/github/gabrielladsc/Deteccao_de_objetos_Python/blob/main/deteccao_de_objetos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalação do Tensorflow, uma biblioteca de machine learning.

In [ ]:
!pip install -U --pre tensorflow=="2.*"
!pip install tf_slim

Instalação do Pycocotools, que é uma API do Python para auxiliar no carregamento, análise e visualização das anotações no COCO.

In [ ]:
!pip install pycocotools

Importando o módulo os, que fornece uma maneira simples de usar funcionalidades que são dependentes de sistema operacional, e o pathlib, para manipular caminhos de sistema de arquivos, independente do sistema operacional.

In [ ]:
import os
import pathlib

Se tiver a parta models no notebook, ele vai acessar ela, senão, irá baixar do github uma pasta com o modelo necessário.

In [ ]:
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

O %%bash significa que os próximos comandos serão executados pelo bash, nesse caso o código vai entrar no caminho especificado e compilar os protos.

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

O %%bash significa que os próximos comandos serão executados pelo bash, nesse caso o código vai entrar no caminho especificado e instalar a API de detecção de objetos.

In [ ]:
%%bash 
cd models/research
pip install .

Fazendo as importações necessárias.

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

Função que carrega dados do site TensorFlow, especificamente os com 'tar.gz' no nome, pois são os modelos necessários.

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

Lista das palavras usadas para escrever o nome de cada objeto nas caixas geradas na detecção.

In [ ]:
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

Mostra pra variável PATH_TO_TEST_IMAGES_DIR em qual caminho buscar as imagens que vão ser processadas para detectar objetos. Além disso especifica qual extensão buscar, podendo ser outras, desde que troque no código.

In [ ]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))

Carrega um modelo de detecção de objetos.

In [ ]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

Função que retorna um dicionário com as informações da detecção. 
Uma imagem é passada como parâmetro e ela deve ser convertida pro tipo tensor.
Como a saída do processamento é um lote de tensores, ela é convertida em matriz numpy para acessar apenas a posição que importa, que é a quantidade de detecções feitas.







In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  
  input_tensor = tf.convert_to_tensor(image)
  
  input_tensor = input_tensor[tf.newaxis,...]

  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  if 'detection_masks' in output_dict:

    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

Função que mostra a imagem com a detecção aplicada. Recebe como parâmetro um modelo de detecção e um caminho para a imagem a ser processada. O caminho da imagem é passado para um array através do numpy. A visualização consiste na imagem em array e os dados do dicionário retornados da função run_inference_for_single_image.

In [ ]:
def show_inference(model, image_path):
  image_np = np.array(Image.open(image_path))

  output_dict = run_inference_for_single_image(model, image_np)
  
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

Esse *for* percorre todas as imagens que estão no caminho dado, com a extensão especificada, passa todas essas imagens como parâmetro pra função show_inference, que imprime as imagens com a detecção aplicada.

In [ ]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)
